# PCA를 이용한 차원 축소

**데이터셋 확인 및 분리**

In [1]:
from sklearn.datasets import fetch_20newsgroups

# 20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# 학습 데이터셋을 가져옴
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),  # 메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로 분류
                                      categories=categories)
# 검증 데이터셋을 가져옴
newsgroups_test = fetch_20newsgroups(subset='test', 
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

In [2]:
X_train = newsgroups_train.data     # 학습 데이터셋 문서
y_train = newsgroups_train.target   # 학습 데이터셋 라벨

X_test = newsgroups_test.data       # 검증 데이터셋 문서
y_test = newsgroups_test.target     # 검증 데이터셋 라벨

**문서 분류를 위한 데이터 준비 (TF-IDF 벡터)**

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
cachedStopWords = stopwords.words("english")
RegTok = RegexpTokenizer("[\w']{3,}")               # 정규포현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english'))     # 영어 불용어를 가져옴

In [8]:
def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    # stopwords 제외
    words = [word for word in tokens if (word not in english_stops) and len(word) > 2]
    # porter stemmer 적용
    features = (list(map(lambda token: PorterStemmer().stem(token),words)))
    return features

In [9]:
tfidf = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf.fit_transform(X_train)    # train set을 변환
X_test_tfidf = tfidf.transform(X_test)          # test set을 변환

**로지스틱 회귀에 대한 성능 차이 확인**

차원 축소 전

In [10]:
from sklearn.linear_model import LogisticRegression 

LR_clf = LogisticRegression()       # 분류기 선언
LR_clf.fit(X_train_tfidf, y_train)  # train data를 이용하여 분류기를 학습

print('Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) 

Train set score: 0.962
Test set score: 0.761


차원 축소 후

In [11]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2000, random_state = 7)    # 차원 축소 20085 -> 2000, reproducible
X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())

print('Original tfidf matrix shape:', X_train_tfidf.shape)
print('PCA Converted matrix shape:', X_train_pca.shape)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum()))    # 축소 전 분산과의 비율 (1이면 축소 전 분산을 모두 설명)

Original tfidf matrix shape: (2034, 20085)
PCA Converted matrix shape: (2034, 2000)
Sum of explained variance ratio: 1.000


In [12]:
LR_clf.fit(X_train_pca, y_train)

print('Train set score: {:.3f}'.format(LR_clf.score(X_train_pca, y_train)))
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test)))

Train set score: 0.962
Test set score: 0.761


**라쏘 회귀와의 성능 차이 확인**

라쏘 회귀를 이용한 특성 선택

In [13]:
import numpy as np

lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=1) 
lasso_clf.fit(X_train_tfidf, y_train) 

print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))

# 계수(coefficient) 중에서 0이 아닌 것들의 개수를 출력
print('#Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1]) 

Train set score: 0.790
Test set score: 0.718
#Used features count: 321 out of 20085


하쏘 회귀에서 선택된 특성의 차원 수와 동일한 차원 수로 축소 후 로지스틱 회귀

In [14]:
pca = PCA(n_components=321, random_state=7)

X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())

print('PCA Converted X shape:', X_train_pca.shape)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum()))

PCA Converted X shape: (2034, 321)
Sum of explained variance ratio: 0.437


In [15]:
LR_clf.fit(X_train_pca, y_train)

print('Train set score: {:.3f}'.format(LR_clf.score(X_train_pca, y_train)))
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test)))

Train set score: 0.875
Test set score: 0.751


차원 수를 획기적으로 줄였을 때 (20085 -> 100)

In [16]:
pca = PCA(n_components=100, random_state=7)

X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())

print('PCA Converted X shape:', X_train_pca.shape)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum()))

PCA Converted X shape: (2034, 100)
Sum of explained variance ratio: 0.211


In [17]:
LR_clf.fit(X_train_pca, y_train)

print('Train set score: {:.3f}'.format(LR_clf.score(X_train_pca, y_train)))
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test)))

Train set score: 0.807
Test set score: 0.738
